In [8]:
version = "v0.1.0"

In [9]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Import neo4j DB: 6/?

Code to translate v2.7.4_PIS-model.xlsx to neo4j database. 

## Setup

In [10]:
from collections import defaultdict

In [11]:
import pandas as pd
import re
import numpy as np
import os

In [12]:
from py2neo import Graph, Node, Relationship

In [13]:
import helpers

In [14]:
from importlib import reload

Connect to graph via docker-compose link. See http://localhost:7474/browser/

In [15]:
graph = Graph(host="neo4j")

In [16]:
from pathlib import Path

base_path = Path("..")
parsed_path = base_path / "data" / "parsed"

## Add string lists for full text search

In [17]:
# str external links

cy = f'''
MATCH (m)
WHERE EXISTS(m.external_links)
SET m._external_links = trim(reduce(s='', el IN m.external_links | s + el + ' '))
'''
print(cy)
graph.run(cy)


MATCH (m)
WHERE EXISTS(m.external_links)
SET m._external_links = trim(reduce(s='', el IN m.external_links | s + el + ' '))



(No data)

In [18]:
# str identifiers
cy = '''
MATCH (m:Family)
WITH m, COALESCE(trim(reduce(s='', el IN m.ath_homologues | s + el + ' ')), '') AS ath,  
        COALESCE(trim(reduce(s='', el IN m.osa_homologues | s + el + ' ')), '') AS osa, 
        COALESCE(trim(reduce(s='', el IN m.sly_homologues | s + el + ' ')), '') AS sly, 
        COALESCE(trim(reduce(s='', el IN m.stu_homologues | s + el + ' ')), '') AS stu
SET m._identifiers = trim(reduce(s='', el IN [ath, osa, sly, stu] | s + el + ' '))
'''
print(cy)
graph.run(cy)


# str identifiers
cy = '''
MATCH (m:FunctionalCluster)
WITH m, COALESCE(trim(reduce(s='', el IN m.ath_homologues | s + el + ' ')), '') AS ath,  
        COALESCE(trim(reduce(s='', el IN m.osa_homologues | s + el + ' ')), '') AS osa, 
        COALESCE(trim(reduce(s='', el IN m.sly_homologues | s + el + ' ')), '') AS sly, 
        COALESCE(trim(reduce(s='', el IN m.stu_homologues | s + el + ' ')), '') AS stu
SET m._identifiers = trim(reduce(s='', el IN [ath, osa, sly, stu] | s + el + ' '))
'''
print(cy)
graph.run(cy)


MATCH (m:Family)
WITH m, COALESCE(trim(reduce(s='', el IN m.ath_homologues | s + el + ' ')), '') AS ath,  
        COALESCE(trim(reduce(s='', el IN m.osa_homologues | s + el + ' ')), '') AS osa, 
        COALESCE(trim(reduce(s='', el IN m.sly_homologues | s + el + ' ')), '') AS sly, 
        COALESCE(trim(reduce(s='', el IN m.stu_homologues | s + el + ' ')), '') AS stu
SET m._identifiers = trim(reduce(s='', el IN [ath, osa, sly, stu] | s + el + ' '))


MATCH (m:FunctionalCluster)
WITH m, COALESCE(trim(reduce(s='', el IN m.ath_homologues | s + el + ' ')), '') AS ath,  
        COALESCE(trim(reduce(s='', el IN m.osa_homologues | s + el + ' ')), '') AS osa, 
        COALESCE(trim(reduce(s='', el IN m.sly_homologues | s + el + ' ')), '') AS sly, 
        COALESCE(trim(reduce(s='', el IN m.stu_homologues | s + el + ' ')), '') AS stu
SET m._identifiers = trim(reduce(s='', el IN [ath, osa, sly, stu] | s + el + ' '))



(No data)

In [19]:
cy = f'''
MATCH (m)
WHERE EXISTS(m.synonyms)
SET m._synonyms = trim(reduce(s='', el IN m.synonyms | s + el + ' '))
'''
print(cy)
graph.run(cy)


MATCH (m)
WHERE EXISTS(m.synonyms)
SET m._synonyms = trim(reduce(s='', el IN m.synonyms | s + el + ' '))



(No data)

## Indexes

In [20]:
# metabolites

cy = '''
CREATE FULLTEXT INDEX  metabolites_all
FOR (m:Metabolite)
ON EACH [
    m.name, 
    m._synonyms, 
    m.description, 
    m.additional_information,    
    m._external_links
]
'''
graph.run(cy)

(No data)

In [21]:
# complexes 

cy = '''
CREATE FULLTEXT INDEX complex_all 
FOR (m:Complex) 
ON EACH [
     m.name, 
     m._synonyms, 
     m.description, 
     m.additional_information,    
     m._external_links
] 
'''
graph.run(cy)

(No data)

In [22]:
# foreign

cy = '''
CREATE FULLTEXT INDEX foreign_all 
FOR (m:Foreign) 
ON EACH [
    m.name, 
    m._synonyms,
    m._identifiers,
    m.description, 
    m.additional_information,    
    m._external_links
    ] 
'''

graph.run(cy)

(No data)

In [23]:
# process

cy = '''
CREATE FULLTEXT INDEX process_all 
FOR (m:Process) 
ON EACH [
    m.name, 
    m._synonyms, 
    m.description, 
    m.additional_information,        
    m._external_links
    ] 
'''
graph.run(cy)

(No data)

In [24]:
# identifiers

cy = '''
CREATE FULLTEXT INDEX family_identifiers 
FOR (m:Family) 
ON EACH [m._identifiers]
'''
graph.run(cy)

(No data)

In [25]:
# plant parts

cy = '''MATCH (n:FunctionalCluster) CALL apoc.create.addLabels(n, ['Plant']) YIELD node RETURN node '''
graph.run(cy)


cy = '''
CREATE FULLTEXT INDEX plant_all 
FOR (m:Plant) 
ON EACH [    
    m.name, 
    m._synonyms, 
    m.description, 
    m.additional_information,    
    m._external_links,
    m._identifiers
    ]
'''
graph.run(cy)

(No data)

In [26]:
# everything 
cy = '''
MATCH (n)
SET n :Node
'''
graph.run(cy)

cy = '''
CREATE FULLTEXT INDEX general_text 
FOR (m:Node)
ON EACH [    
    m.name, 
    m._synonyms, 
    m.description, 
    m.additional_information,
    m._external_links,
    m._identifiers
    ]
'''
graph.run(cy)

(No data)

## Data Science graph

In [27]:
cy = '''
CALL gds.graph.create.cypher(
    'reaction-graph1',
    'MATCH (n) WHERE NOT n:Family RETURN id(n) AS id',
    'MATCH (a)-[r]->(b) WHERE EXISTS(r.reaction_type) RETURN id(a) AS source, id(b) AS target'
)
YIELD graphName, nodeCount, relationshipCount, createMillis;
'''
graph.run(cy)

 graphName       | nodeCount | relationshipCount | createMillis 
-----------------|-----------|-------------------|--------------
 reaction-graph1 |       874 |              1176 |         1312 

In [28]:
# check connected components

cy = '''
CALL gds.wcc.stream('reaction-graph1')
YIELD nodeId, componentId
WITH componentId, count(*) AS componentSize
RETURN componentId, componentSize ORDER BY componentId
'''
data = graph.run(cy).data()
cc_sizes = pd.DataFrame(data)
cc_sizes

,componentId,componentSize
0,0,721
1,3,13
2,49,1
3,59,1
4,63,1
5,65,1
6,66,1
7,71,1
8,74,1
9,77,1


In [29]:
cy = '''
CALL gds.wcc.stream('reaction-graph1')
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).name AS name, componentId
ORDER BY componentId, name
'''
data = graph.run(cy).data()
cc_nodes = pd.DataFrame(data)
cc_nodes

,name,componentId
0,"12,13-EOT",0
1,12-OH-JA-Ile,0
2,13-HPOT,0
3,"AAO[AT1G04580,AT2G27150,AT5G20960]",0
4,ACC,0
...,...,...
869,rx00306,297
870,NDR1[AT3G20600],306
871,NDR1|RIN4,306
872,RIN4[AT3G25070],306


In [30]:
for componentID, subdf in cc_nodes.groupby('componentId'):
    print("compponentID")
    if subdf.shape[0] < cc_sizes['componentSize'].max():
        print(f'{componentID}\t{subdf.shape[0]}\t{", ".join(list(subdf["name"].values))}')

compponentID
compponentID
3	13	9-cis-&beta;-carotene, 9-cis-10&prime;-apo-&beta;-carotenal, CCD7[AT2G44990], CCD8[AT4G32810], CL, CLA, D27[OS11G0587000], MAX1[AT2G26170], all-trans-&beta;-carotene, rx00314, rx00315, rx00316, rx00317
compponentID
49	1	NADPH
compponentID
59	1	H2O
compponentID
63	1	HO2.
compponentID
65	1	O3
compponentID
66	1	OH.
compponentID
71	1	SL
compponentID
74	1	UDP glucose
compponentID
77	1	anthocyanin
compponentID
107	1	tZRP
compponentID
110	1	DAMP/HAMP
compponentID
111	1	PostROS
compponentID
112	1	PreROS
compponentID
116	1	elf18
compponentID
118	1	ch
compponentID
119	1	6K1
compponentID
120	1	6K2
compponentID
122	8	CP, CPIP1,2b[AT1G10350], CPIP2a[AT3G08910], CP|CPIP1,2b, CP|CPIP2a, rx00201, rx00202, rx00203
compponentID
124	1	NIa-Pro
compponentID
125	1	NIb
compponentID
126	1	P1
compponentID
128	1	P3N-PIPO
compponentID
129	16	CO[AT5G15840], CO|OBE1, OBE1[AT3G07780], OBE1|VPg, OBE1|WRKY11, OBE1|WRKY17, RH8[AT4G00660], RH8|VPg, VPg, WRKY11[AT4G31550], WRKY17[AT2G24570

# END 